In [1]:
import pandas as pd
import numpy as np

In [2]:
sp500_futures = pd.read_csv("../data/sp500_futures_tick_data.csv")
sp500_futures.head()

,date,time,price,volume
0,01/03/2000,08:30:34.000,1496.4,0
1,01/03/2000,08:30:36.000,1496.0,0
2,01/03/2000,08:30:37.000,1495.5,0
3,01/03/2000,08:30:46.000,1495.0,0
4,01/03/2000,08:30:53.000,1495.5,0


In [3]:
print(sp500_futures.date.min())
print(sp500_futures.date.max())

01/01/2015
12/31/2019


## 0. Add SP500 spot price

Join the price of sp500 to sp500_futures with matching date

In [4]:
sp500_original = pd.read_csv('../data/sp500.csv')
# The time is always "00:00:00"
sp500 = pd.DataFrame()
# Remove the time, which is always 00:00:00
sp500['date'] = sp500_original.Date.str.replace("00:00:00", "").str.strip()
# Change date format to match with dates from sp500_futures
sp500['date'] = pd.to_datetime(sp500['date']).dt.strftime('%m/%d/%Y')
# Set the price using the adjusted close
sp500['price'] = sp500_original['Adj Close']

del sp500_original

sp500.head()

,date,price
0,01/03/1950,16.66
1,01/04/1950,16.85
2,01/05/1950,16.93
3,01/06/1950,16.98
4,01/09/1950,17.08


In [5]:
sp500_futures = pd.merge(sp500_futures, sp500, on='date') # inner join by default
sp500_futures.rename(columns={'price_x': 'price', 'price_y': 'spot_price'}, inplace=True)
sp500_futures.head()

,date,time,price,volume,spot_price
0,01/03/2000,08:30:34.000,1496.4,0,1455.22
1,01/03/2000,08:30:36.000,1496.0,0,1455.22
2,01/03/2000,08:30:37.000,1495.5,0,1455.22
3,01/03/2000,08:30:46.000,1495.0,0,1455.22
4,01/03/2000,08:30:53.000,1495.5,0,1455.22


## 1. Standard bars

### Convert time bars to tick bars

Note: Pick timestamp when the information was fully available to avoid look-ahead effects

Collect all time bars for each day
Get the bars for every tick count: date, time, bar_count, open, close, high, low, total_volume, complete

1. Is there any pre-hour and post-hour trading activities?
Yes, find the min and max of column 'time'

2. How to handle incomplete tick bars?
Answer: Keep the count of time bars, and add a column named 'complete' to mark if the tick bar is complete or not

3. Should I cutoff tick bars for each day? How to handle overnight trading?
Answer: For now, let's consider each day from 00:00 to 23:59 as a whole trading day

In [6]:
tick_size = 1000
grouped_by_date = sp500_futures.groupby('date')
# Within each date group, dividing further into each sub-group of `tick_size' rows
sp500_futures['group_in_date'] = grouped_by_date.cumcount() // tick_size

In [7]:
def aggregate_bars(x):
    d = {}
    d['time'] = x['time'].iloc[-1]
    d['open'] = x['price'].iloc[0]
    d['close'] = x['price'].iloc[-1]
    d['low'] = x['price'].min()
    d['high'] = x['price'].max()
    d['spot_price'] = x['spot_price']
    d['total_volume'] = x['volume'].sum()
    # In edge case, d['total_volume'] can be 0 if all volume in the df is 0
    d['vol_weighted_avg_price'] = x['price'].dot(x['volume']) / d['total_volume'] if d['total_volume'] else 0
    d['count'] = x['date'].count()
    return pd.Series(d)

In [8]:
sp500_futures_ticks = sp500_futures.groupby(['date', 'group_in_date'], as_index=False).apply(aggregate_bars)
sp500_futures_ticks['complete'] = sp500_futures_ticks['count'] == tick_size
sp500_futures_ticks.head()

,date,group_in_date,time,open,close,low,high,spot_price,total_volume,vol_weighted_avg_price,count,complete
0,01/02/2001,0,10:30:40.000,1330.0,1312.5,1307.5,1334.8,837355 1283.27 837356 1283.27 837357 ...,0,0.0,1000,True
1,01/02/2001,1,12:46:56.000,1312.4,1307.0,1305.0,1315.0,838355 1283.27 838356 1283.27 838357 ...,0,0.0,1000,True
2,01/02/2001,2,14:55:41.000,1307.0,1291.5,1289.0,1309.5,839355 1283.27 839356 1283.27 839357 ...,0,0.0,1000,True
3,01/02/2001,3,15:14:36.000,1291.0,1299.4,1291.0,1301.0,840355 1283.27 840356 1283.27 840357 ...,0,0.0,151,False
4,01/02/2002,0,10:08:00.000,1150.0,1141.2,1136.5,1150.5,1612253 1154.67 1612254 1154.67 1612255 ...,0,0.0,1000,True


### Convert time bars to volume bars

In [9]:
volume_size = 1000
sp500_futures['volume_group'] = sp500_futures.volume.cumsum() // volume_size

In [10]:
sp500_futures_volumes = sp500_futures.groupby('volume_group', as_index=False).apply(aggregate_bars)
sp500_futures_volumes.head()

,volume_group,time,open,close,low,high,spot_price,total_volume,vol_weighted_avg_price,count
0,0,03:13:12.000,1496.4,971.8,767.5,1574.0,0 1455.22 1 1455.22 2 ...,997,973.188365,2813357
1,1,06:27:39.000,971.9,971.1,971.1,973.2,2813357 982.32 2813358 982.32 2813359 ...,999,972.021121,524
2,2,07:43:08.000,971.3,968.3,967.6,972.0,2813881 982.32 2813882 982.32 2813883 ...,1003,970.340678,451
3,3,15:56:37.000,968.0,981.6,960.5,982.4,2814332 982.32 2814333 982.32 2814334 ...,1000,969.736700,3448
4,4,01:26:16.000,981.7,982.5,981.3,984.0,2817780 982.32 2817781 982.32 2817782 ...,999,982.660761,416


### Convert time bars to dollar bars

In [11]:
dollar_size = 100000
sp500_futures['dollar'] = sp500_futures.price * sp500_futures.volume
sp500_futures['dollar_group'] = sp500_futures['dollar'].cumsum().astype(int) // dollar_size

In [12]:
sp500_futures.volume_group.describe()

count    1.347623e+07
mean     4.422480e+03
std      3.667501e+03
min      0.000000e+00
25%      5.660000e+02
50%      4.056000e+03
75%      7.788000e+03
max      1.112800e+04
Name: volume_group, dtype: float64

In [13]:
def aggregate_dollar_bars(x):
    d = {}
    d['time'] = x['time'].iloc[-1]
    d['open'] = x['price'].iloc[0]
    d['close'] = x['price'].iloc[-1]
    d['low'] = x['price'].min()
    d['high'] = x['price'].max()
    d['spot_price'] = x['spot_price']
    d['total_volume'] = x['volume'].sum()
    # In edge case, d['total_volume'] can be 0 if all volume in the df is 0
    d['vol_weighted_avg_price'] = x['price'].dot(x['volume']) / d['total_volume'] if d['total_volume'] else 0
    d['dollar'] = x['dollar'].sum()
    d['count'] = x['date'].count()
    return pd.Series(d)

In [14]:
sp500_futures_dollar = sp500_futures.groupby('dollar_group', as_index=False).apply(aggregate_dollar_bars)
sp500_futures_dollar.head()

,dollar_group,time,open,close,low,high,spot_price,total_volume,vol_weighted_avg_price,dollar,count
0,0,23:04:34.000,1496.4,972.0,767.5,1574.0,0 1455.22 1 1455.22 2 ...,12,971.991667,11663.9,2812948
1,1,00:11:37.000,972.0,972.0,971.5,972.6,2812948 974.50 2812949 974.50 2812950 ...,186,972.025269,180796.7,31
2,2,00:47:02.000,972.0,972.7,972.0,972.7,2812979 982.32 2812980 982.32 2812981 ...,87,972.313793,84591.3,19
3,3,01:33:16.000,972.8,974.0,972.8,974.0,2812998 982.32 2812999 982.32 2813000 ...,109,973.339450,106094.0,32
4,4,02:04:52.000,974.0,974.6,974.0,974.6,2813030 982.32 2813031 982.32 2813032 ...,118,974.216102,114957.5,55


## 2. Information-driven bars

In [15]:
sp500_futures['next_price'] = sp500_futures.price.shift(-1)

In [16]:
sp500_futures['delta_p'] = sp500_futures.next_price - sp500_futures.price
sp500_futures.dropna(inplace=True)

# Initialize to 0s
sp500_futures['tick_b_t'] = np.where(sp500_futures['delta_p'] == 0, 0, np.sign(sp500_futures['delta_p']))
sp500_futures.head()

,date,time,price,volume,spot_price,group_in_date,volume_group,dollar,dollar_group,next_price,delta_p,tick_b_t
0,01/03/2000,08:30:34.000,1496.4,0,1455.22,0,0,0.0,0,1496.0,-0.4,-1.0
1,01/03/2000,08:30:36.000,1496.0,0,1455.22,0,0,0.0,0,1495.5,-0.5,-1.0
2,01/03/2000,08:30:37.000,1495.5,0,1455.22,0,0,0.0,0,1495.0,-0.5,-1.0
3,01/03/2000,08:30:46.000,1495.0,0,1455.22,0,0,0.0,0,1495.5,0.5,1.0
4,01/03/2000,08:30:53.000,1495.5,0,1455.22,0,0,0.0,0,1495.0,-0.5,-1.0


## 3. ETF Rolling